-------------------------------------------
# **WEB SCRAPPING CON BeautifulSoup-Selenium**
___________________________________________

![Texto alternativo](ws1.PNG)

## **Ficheros incluidos en la Página Web (URIs, URLs y URNs)**




> *`URIs` --->   Uniform Resource Identifier o identificador de recursos uniforme) es un nombre, o formalmente una cadena de caracteres, que identifica un recurso de forma accesible dentro de una red: una página web, un fichero, etc. Su forma genérica es:

        Schema:[//[user[:passwd]@]host[:port]][/path][?query][#tag]

* (//[usuario[:passwd]@]host[:port]) ----> se la conoce como autoridad
                    
* ruta (path) ---> es una secuencia de nombres separadas  ya sea por los símbolos “/” o “:”

* consulta (query) y una etiqueta (tag) ----> para acceder a un lugar concreto del documento

Ejemplo: https://es.wikipedia.org/wiki/Alan_Turing#Turing_en_el_cine

Por ejemplo https://www.youtube.com/watch?v=iSh9qg-2qKw. En este caso, la consulta se incluye con la forma v=codificaciónDeLaConsulta.
                

>*`URLs` --->   s (Uniform Resource Locator o localizador de recurso uniforme), a menudo conocidas simplemente como direcciones web, sirven para especificar un recurso en la red mediante su localización y una forma o protocolo que permite recuperarlo, que corresponde con la parte schema de la estructura general de la URI. 


Además de esquemas o protocolos http o https, en ocasiones encontraremos URLs que especifican otros, como el File Transfer Protocol (FTP), por ejemplo en la siguiente URL:                 ftp://example.com.

>*`URNs` --->  (Uniform Resource Name, o nombre de recurso uniforme), el otro tipo de URIs, identifica un recurso, pero no tienen por qué incluir ninguna forma de localizarlos. Un ejemplo puede ser urn:isbn:0-391-31341-0.

## Ejemplo: datos de contaminación en Madrid

    http://www.mambiente.munimadrid.es/opendata/horario.txt
Esta URL proporciona los datos de las estaciones de medición de la calidad del aire
de la ciudad de Madrid. Supongamos que queremos descargar el fichero
correspondiente para analizarlo a continuación. Para ello, nos bastará con incluir la
biblioteca requests que incluye un método para “pedir” (get) el fichero, que
podemos grabar a continuación en nuestro disco duro local:

In [2]:
import requests

In [3]:
url = "https://datos.madrid.es/egob/catalogo/218490-244-catalogo-datasets.txt"
resp = requests.get(url)
print(resp) #Response 200 indica que se descargó con éxito en la URL

<Response [200]>


Distintos status_code, los que empiezan con 2 suelen indicar éxito, mientras que los que comienzan por 4 o 5 indican error, como el famosísimo 404 (recurso no encontrado) o el 403 (acceso prohibido al recurso).

In [4]:
path = 'Web_Scrapping_RafaelCaballero'
with open (path + 'catalogo.txt', 'wb') as output: 
    output.write(resp.content)

In [5]:
import matplotlib.pyplot as plt
import csv

In [6]:
import requests
from contextlib import closing
import csv
import codecs
import matplotlib.pyplot as plt

In [7]:
url =  "https://datos.madrid.es/egob/catalogo/218490-244-catalogo-datasets.txt"
with closing(requests.get(url, stream = True)) as r: 
    reader = csv.reader(codecs.iterdecode(
                            r.iter_lines(),
                            'utf-8'),
                            delimiter=',')
for row in reader: 
    print(row)
    break
    

In [8]:
display(reader)

In [11]:
import requests
import csv
import chardet


url = "https://datos.madrid.es/egob/catalogo/218490-244-catalogo-datasets.txt"


local_filename = 'catalogo-datasets.txt'


response = requests.get(url)
with open(local_filename, 'wb') as f:
    f.write(response.content)


with open(local_filename, 'rb') as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']


with open(local_filename, 'r', encoding=encoding) as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')  # Using tab delimiter as a guess
    for row in reader:
        print(row)
        break  #Imprime solo la 1ra columna 


['FechaDescarga', 'idDataSet', 'Nombre', 'sector', 'Número de recursos', 'Descargas', 'Número de votos', 'formato_salida', 'frecuencia', '22/11/2018', 'F_ultima_act_metadatos', 'URL', 'FechaDescarga']


## Datos que forman parte de la página 

Página web, que normalmente está compuesta por código HTML, imágenes,scripts y ficheros de estilo. 

Los datos que forman parte de la página pueden ser:

* Indica que se trata de un documento HTML estándar de las páginas web -->  <``!DOCTYPE html``>

* Etiqueta que marca el comienzo del documento, y que se cerrará al final
del documento -->  <`/html`>  <`/html`>

* , los elementos tienen una estructura con la forma:
<`elemento atributo="valor">Contenido</elemento`>

* La cabecera del documento, como puede ser su título, el autor -->  <`head> … </head`>

* Es el contenido en sí de la página, que es lo que nosotros deseamos examinar --> <`body> … </body`>



Dentro del elemento body habrá otros elementos, que dependerán de la página, que a su vez pueden contener otros elementos, y así sucesivamente. En nuestro pequeño ejemplo el cuerpo solo tiene dos frases, marcadas por las etiquetas <`div`> y <`/div`>. Podemos grabar este código en un fichero con el nombre que deseemos, por ejemplo mini.html. Haciendo doble clic sobre el fichero se abrirá el navegador y veremos la (humilde) página

In [15]:
""" from bs4 import BeautifulSoup
url = r"c:\...\mini.html"
with open(url, "r") as f:
    page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    print(soup.prettify()) """

' from bs4 import BeautifulSoup\nurl = r"c:\\...\\mini.html"\nwith open(url, "r") as f:\n    page = f.read()\n    soup = BeautifulSoup(page, "html.parser")\n    print(soup.prettify()) '

## Navegación absoluta

HTML siguen una estructura jerárquica. Los elementos que están directamente dentro de otros se dice que son hijos del elemento contenedor y hermanos entre sí. En nuestro miniejemplo, los elementos <`div id="date"`> y <`div id="content"`> son hermanos entre sí, y ambos son hijos del elemento `<body`>. Además, <`div id="date"`> tiene a su vez un hijo, que en esta ocasión no es otro elemento sino un texto.


La biblioteca BeautifulSoup nos permite utilizar estos conceptos para navegar por el documento buscando la información que precisamos. Como ejemplo inicial, podemos preguntar por los hijos del documento raíz, y su tipo para hacernos una idea de lo que vamos a encontrarnos:

In [17]:
""" hijosDoc = list(soup.children)
print([type(item) for item in hijosDoc])
print(hijosDoc) """

' hijosDoc = list(soup.children)\nprint([type(item) for item in hijosDoc])\nprint(hijosDoc) '

## Navegación relativa 

BeautifulSoup ofrece la posibilidad de buscar elementos sin tener
que explicar la ruta completa. Por ejemplo, el método find_all() busca todas las apariciones de un elemento a cualquier nivel y las devuelve en forma de lista:
`divs = soup.find_all("div")`

Ahora, si sabemos que queremos mostrar el texto asociado al primer div
podemos escribir:
`print(divs[0].get_text())`

Mediante el método find, devuelve directamente el primer objeto del
documento que representa el elemento buscado:
`print(soup.find("div").get_text())`

### Ejemplo: día y hora oficiales 

In [18]:
import requests
url = "https://www.boe.es/sede_electronica/informacion/hora_oficial.php" 
r = requests.get(url)
print(r)

<Response [404]>


In [2]:
import requests # pip install requests
from lxml import html # pip install lxml

# USER AGENT PARA PROTEGERNOS DE BANEOS
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

# URL SEMILLA
url = 'https://www.wikipedia.org/'

# REQUERIMIENTO AL SERVIDOR
respuesta = requests.get(url, headers=headers)
respuesta.encoding = 'utf-8' # Codificar correctamente caracteres extranos

# PARSEO DEL ARBOL HTML QUE RECIBO COMO RESPUESTA CON LXML
parser = html.fromstring(respuesta.content) # Uso .content para poder codificar los caracteres raros

# EXTRACCION DE IDIOMA INGLES
ingles = parser.get_element_by_id("js-link-box-en")
print (ingles.text_content())

# EXTRACCION SOLO DEL TEXTO QUE DICE INGLES
ingles = parser.xpath("//a[@id='js-link-box-en']/strong/text()")
print(ingles[0])

# EXTRACCION DE TODOS LOS IDIOMAS POR CLASE
idiomas = parser.find_class('central-featured-lang')
for idioma in idiomas:
  print(idioma.text_content())

# EXTRACCION DE TODOS LOS IDIOMAS POR XPATH
idiomas = parser.xpath("//div[contains(@class,'central-featured-lang')]//strong/text()")
for idioma in idiomas:
  print(idioma)

2024-06-14 19:58:20 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.wikipedia.org:443
2024-06-14 19:58:21 [urllib3.connectionpool] DEBUG: https://www.wikipedia.org:443 "GET / HTTP/1.1" 200 22499



English
6,796,000+ articles

English


English
6,796,000+ articles




日本語
1,407,000+ 記事




Español
1.938.000+ artículos




Русский
1 969 000+ статей




Deutsch
2.891.000+ Artikel




Français
2 598 000+ articles




中文
1,409,000+ 条目 / 條目




Italiano
1.853.000+ voci




فارسی
۹۹۵٬۰۰۰+ مقاله




Português
1.120.000+ artigos


English
日本語
Español
Русский
Deutsch
Français
中文
Italiano
Português


In [4]:
import requests
from bs4 import BeautifulSoup # pip install beautifulsoup4

# USER AGENT PARA PROTEGERNOS DE BANEOS
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# URL SEMILLA
url = 'https://stackoverflow.com/questions'

# REQUERIMIENTO AL SERVIDOR
respuesta = requests.get(url, headers=headers)

# PARSEO DEL ARBOL CON BEAUTIFUL SOUP
soup = BeautifulSoup(respuesta.text)
contenedor_de_preguntas = soup.find(id="questions") # ENCONTRAR UN ELEMENTO POR ID
lista_de_preguntas = contenedor_de_preguntas.find_all('div', class_="s-post-summary") # ENCONTRAR VARIOS ELEMENTOS POR TAG Y POR CLASE
for pregunta in lista_de_preguntas: # ITERAR ELEMENTO POR ELEMENTO

  # METODO #1: METODO TRADICIONAL
  texto_pregunta = pregunta.find('h3').text # DENTRO DE CADA ELEMENTO ITERADO ENCONTRAR UN TAG
  descripcion_pregunta = pregunta.find(class_='s-post-summary--content-excerpt').text # ENCONTRAR POR CLASE
  descripcion_pregunta = descripcion_pregunta.replace('\n', '').replace('\r', '') # LIMPIEZA DE TEXTO
  print (texto_pregunta)
  print (descripcion_pregunta)
  print ()


  # METODO #2: APROVECHANDO EL PODER COMPLETO DE BEAUTIFUL SOUP
  contenedor_pregunta = pregunta.find('h3')
  texto_pregunta = contenedor_pregunta.text
  descripcion_pregunta = contenedor_pregunta.find_next_sibling('div') # TRAVERSANDO EL ARBOL DE UNA MENERA DIFERENTE
  texto_descripcion_pregunta = descripcion_pregunta.text

  texto_descripcion_pregunta = texto_descripcion_pregunta.replace('\n', '').replace('\t', '')
  print (texto_pregunta)
  print (texto_descripcion_pregunta)
  print ()

2024-06-14 20:01:20 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): stackoverflow.com:443
2024-06-14 20:01:21 [urllib3.connectionpool] DEBUG: https://stackoverflow.com:443 "GET /questions HTTP/1.1" 200 None



Stereoscopic video player web

                I have an side by side video with right and left eye perspective in one single mp4 file. I want to be able to play it on an website. It should be shown in vr. The video should be splitted and rendered ...            


Stereoscopic video player web

                I have an side by side video with right and left eye perspective in one single mp4 file. I want to be able to play it on an website. It should be shown in vr. The video should be splitted and rendered ...            


Which PCA results are correct?

                I aim to find which directions in my data have "vary greatly". To do that, I understand a method called PCA, which uses the eigenvectors of the covariant matrix to find them.I used ...            


Which PCA results are correct?

                I aim to find which directions in my data have "vary greatly". To do that, I understand a method called PCA, which uses the eigenvectors of the covariant matrix to find the